In [143]:
import pandas as pd
import json
from ast import literal_eval

# Clean incorrect responses from GPT4

In [144]:
with open('../datasets/gpt4/arc_challenge_train_gpt4_response_parsed.json', 'r') as f:
    arc_parsed_data = json.load(f)
len(arc_parsed_data)

1000

In [145]:
with open('../datasets/gpt4/winogrande_train_gpt4_response_parsed.json', 'r') as f:
    wino_parsed_data = json.load(f)
len(wino_parsed_data)

1000

In [146]:
arc_df = pd.read_csv('../datasets/arc_challenge_train_with_prompt.csv')
arc_df.head()

,id,question,choices,answerKey,options,prompt
0,Mercury_SC_415702,George wants to warm his hands quickly by rubb...,"{'text': array(['dry palms', 'wet palms', 'pal...",A,"['dry palms', 'wet palms', 'palms covered with...",Question: - George wants to warm his hands qui...
1,MCAS_2009_5_6516,Which of the following statements best explain...,{'text': array(['The refrigerator door is smoo...,B,"['The refrigerator door is smooth.', 'The refr...",Question: - Which of the following statements ...
2,Mercury_7233695,A fold observed in layers of sedimentary rock ...,"{'text': array(['cooling of flowing magma.', '...",B,"['cooling of flowing magma.', 'converging of c...",Question: - A fold observed in layers of sedim...
3,Mercury_7041615,Which of these do scientists offer as the most...,"{'text': array(['worldwide disease', 'global m...",D,"['worldwide disease', 'global mountain buildin...",Question: - Which of these do scientists offer...
4,Mercury_7041860,A boat is acted on by a river current flowing ...,"{'text': array(['west', 'east', 'north', 'sout...",B,"['west', 'east', 'north', 'south']",Question: - A boat is acted on by a river curr...


In [147]:
wino_df = pd.read_csv('../datasets/winogrande_train_with_prompt.csv')
wino_df.head()

,question,option1,option2,answer,prompt,id
0,John moved the couch from the garage to the ba...,garage,backyard,1,Question: - John moved the couch from the gara...,GO3wkLp9TC
1,The doctor diagnosed Justin with bipolar and R...,Justin,Robert,2,Question: - The doctor diagnosed Justin with b...,IPQZOkoLLz
2,Dennis drew up a business proposal to present ...,Dennis,Logan,1,Question: - Dennis drew up a business proposal...,W85KDUn8gA
3,Felicia unexpectedly made fried eggs for break...,Felicia,Katrina,2,Question: - Felicia unexpectedly made fried eg...,UIUCGI68OY
4,My shampoo did not lather easily on my Afro ha...,shampoo,hair,2,Question: - My shampoo did not lather easily o...,WFAUCgvz91


In [148]:
col = wino_df.pop('id')
wino_df.insert(0, col.name, col)
wino_df.head()

,id,question,option1,option2,answer,prompt
0,GO3wkLp9TC,John moved the couch from the garage to the ba...,garage,backyard,1,Question: - John moved the couch from the gara...
1,IPQZOkoLLz,The doctor diagnosed Justin with bipolar and R...,Justin,Robert,2,Question: - The doctor diagnosed Justin with b...
2,W85KDUn8gA,Dennis drew up a business proposal to present ...,Dennis,Logan,1,Question: - Dennis drew up a business proposal...
3,UIUCGI68OY,Felicia unexpectedly made fried eggs for break...,Felicia,Katrina,2,Question: - Felicia unexpectedly made fried eg...
4,WFAUCgvz91,My shampoo did not lather easily on my Afro ha...,shampoo,hair,2,Question: - My shampoo did not lather easily o...


## Extracting answer

In [149]:
key_to_answer = {'A' : 0, 'B' : 1, 'C' : 2, 'D' : 3, 'E': 4}

In [150]:
def get_answer_arc(row):
    question = row['question']
    options = row['options']
    list_options = literal_eval(options)
    try:
        answer = list_options[key_to_answer[row['answerKey']]]
    except:
        answer = list_options[int(row['answerKey'])-1]
    return answer.strip().lower()

In [151]:
arc_df['answer_text'] = arc_df.apply(lambda row: get_answer_arc(row), axis=1)
arc_df.head()

,id,question,choices,answerKey,options,prompt,answer_text
0,Mercury_SC_415702,George wants to warm his hands quickly by rubb...,"{'text': array(['dry palms', 'wet palms', 'pal...",A,"['dry palms', 'wet palms', 'palms covered with...",Question: - George wants to warm his hands qui...,dry palms
1,MCAS_2009_5_6516,Which of the following statements best explain...,{'text': array(['The refrigerator door is smoo...,B,"['The refrigerator door is smooth.', 'The refr...",Question: - Which of the following statements ...,the refrigerator door contains iron.
2,Mercury_7233695,A fold observed in layers of sedimentary rock ...,"{'text': array(['cooling of flowing magma.', '...",B,"['cooling of flowing magma.', 'converging of c...",Question: - A fold observed in layers of sedim...,converging of crustal plates.
3,Mercury_7041615,Which of these do scientists offer as the most...,"{'text': array(['worldwide disease', 'global m...",D,"['worldwide disease', 'global mountain buildin...",Question: - Which of these do scientists offer...,impact of an asteroid created dust that blocke...
4,Mercury_7041860,A boat is acted on by a river current flowing ...,"{'text': array(['west', 'east', 'north', 'sout...",B,"['west', 'east', 'north', 'south']",Question: - A boat is acted on by a river curr...,east


In [152]:
def get_answer_wino(row):
    answer = row['option1']
    if row['answer'] == 2:
        answer = row['option2']
    return answer.strip().lower()

In [153]:
wino_df['answer_text'] = wino_df.apply(lambda row: get_answer_wino(row), axis=1)
wino_df.head()

,id,question,option1,option2,answer,prompt,answer_text
0,GO3wkLp9TC,John moved the couch from the garage to the ba...,garage,backyard,1,Question: - John moved the couch from the gara...,garage
1,IPQZOkoLLz,The doctor diagnosed Justin with bipolar and R...,Justin,Robert,2,Question: - The doctor diagnosed Justin with b...,robert
2,W85KDUn8gA,Dennis drew up a business proposal to present ...,Dennis,Logan,1,Question: - Dennis drew up a business proposal...,dennis
3,UIUCGI68OY,Felicia unexpectedly made fried eggs for break...,Felicia,Katrina,2,Question: - Felicia unexpectedly made fried eg...,katrina
4,WFAUCgvz91,My shampoo did not lather easily on my Afro ha...,shampoo,hair,2,Question: - My shampoo did not lather easily o...,hair


## Removing rows with incorrect answer

In [154]:
def check_answer(id_, answer_text, dataset='arc'):
    if dataset == 'arc':
        parsed_data = arc_parsed_data
    else:
        parsed_data = wino_parsed_data
    if id_ not in parsed_data:
        return False
    data = parsed_data[id_]
    # print(id_)
    model_answer = data['parsed_response']['final_answer']['answer'].strip().lower()
    if model_answer == answer_text:
        return True
    return False

In [155]:
len(arc_df)

1119

In [156]:
wrong_answer_filter = arc_df.apply(lambda row: check_answer(row.id, row.answer_text), axis=1)
arc_df_cleaned = arc_df[wrong_answer_filter]

In [157]:
wrong_answer_filter = wino_df.apply(lambda row: check_answer(row.id, row.answer_text, dataset='wino'), axis=1)
wino_df_cleaned = wino_df[wrong_answer_filter]

In [158]:
len(arc_df_cleaned), len(wino_df_cleaned)

(940, 807)

## Adding gpt4 responses

In [159]:
preferred_response_tmplt = """All the main real world concepts essential to answer the question are: 
{}

In terms of ontology, the relationship for each concept to the other concepts in the context of the sentence and the question are:
{}

Answer to the original question and the reasoning for it based on the concepts/entities and their relationships I have identified is:
{}
"""

In [160]:
def get_gpt4_response(id_, dataset='arc'):
    if dataset == 'arc':
        parsed_data = arc_parsed_data
    else:
        parsed_data = wino_parsed_data

    concepts = json.dumps({"concepts":parsed_data[id_]['parsed_response']['concepts']}, indent=2)
    relationships = json.dumps({"relationships":parsed_data[id_]['parsed_response']['relationships']}, indent=2)
    answer = json.dumps(parsed_data[id_]['parsed_response']['final_answer'], indent=2)

    return preferred_response_tmplt.format(concepts, relationships, answer)

In [161]:
arc_df_cleaned['chosen'] = arc_df_cleaned['id'].apply(lambda x: get_gpt4_response(x))
arc_df_cleaned.head()

/var/folders/br/9zwysktd11s2ljdm2_9z0mtm0000gn/T/ipykernel_68779/3395922835.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arc_df_cleaned['chosen'] = arc_df_cleaned['id'].apply(lambda x: get_gpt4_response(x))


,id,question,choices,answerKey,options,prompt,answer_text,chosen
0,Mercury_SC_415702,George wants to warm his hands quickly by rubb...,"{'text': array(['dry palms', 'wet palms', 'pal...",A,"['dry palms', 'wet palms', 'palms covered with...",Question: - George wants to warm his hands qui...,dry palms,All the main real world concepts essential to ...
1,MCAS_2009_5_6516,Which of the following statements best explain...,{'text': array(['The refrigerator door is smoo...,B,"['The refrigerator door is smooth.', 'The refr...",Question: - Which of the following statements ...,the refrigerator door contains iron.,All the main real world concepts essential to ...
2,Mercury_7233695,A fold observed in layers of sedimentary rock ...,"{'text': array(['cooling of flowing magma.', '...",B,"['cooling of flowing magma.', 'converging of c...",Question: - A fold observed in layers of sedim...,converging of crustal plates.,All the main real world concepts essential to ...
3,Mercury_7041615,Which of these do scientists offer as the most...,"{'text': array(['worldwide disease', 'global m...",D,"['worldwide disease', 'global mountain buildin...",Question: - Which of these do scientists offer...,impact of an asteroid created dust that blocke...,All the main real world concepts essential to ...
4,Mercury_7041860,A boat is acted on by a river current flowing ...,"{'text': array(['west', 'east', 'north', 'sout...",B,"['west', 'east', 'north', 'south']",Question: - A boat is acted on by a river curr...,east,All the main real world concepts essential to ...


In [162]:
wino_df_cleaned['chosen'] = wino_df_cleaned['id'].apply(lambda x: get_gpt4_response(x, dataset='wino'))
wino_df_cleaned.head()

/var/folders/br/9zwysktd11s2ljdm2_9z0mtm0000gn/T/ipykernel_68779/1586227962.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wino_df_cleaned['chosen'] = wino_df_cleaned['id'].apply(lambda x: get_gpt4_response(x, dataset='wino'))


,id,question,option1,option2,answer,prompt,answer_text,chosen
0,GO3wkLp9TC,John moved the couch from the garage to the ba...,garage,backyard,1,Question: - John moved the couch from the gara...,garage,All the main real world concepts essential to ...
1,IPQZOkoLLz,The doctor diagnosed Justin with bipolar and R...,Justin,Robert,2,Question: - The doctor diagnosed Justin with b...,robert,All the main real world concepts essential to ...
2,W85KDUn8gA,Dennis drew up a business proposal to present ...,Dennis,Logan,1,Question: - Dennis drew up a business proposal...,dennis,All the main real world concepts essential to ...
3,UIUCGI68OY,Felicia unexpectedly made fried eggs for break...,Felicia,Katrina,2,Question: - Felicia unexpectedly made fried eg...,katrina,All the main real world concepts essential to ...
4,WFAUCgvz91,My shampoo did not lather easily on my Afro ha...,shampoo,hair,2,Question: - My shampoo did not lather easily o...,hair,All the main real world concepts essential to ...


In [163]:
retained_arc_keys = {id_: True for id_ in arc_df_cleaned.id.values.tolist()}
retained_wino_keys = {id_: True for id_ in wino_df_cleaned.id.values.tolist()}

In [164]:
len(retained_arc_keys), len(retained_wino_keys)

(940, 807)

## Adding mistral responses

### ARC

In [165]:
arc_mistral_df = pd.read_csv('../datasets/mistral/arc_train_with_prompt_and_response.csv')
arc_mistral_df.head()

,id,question,choices,answerKey,options,prompt,prompt_response
0,Mercury_SC_415702,George wants to warm his hands quickly by rubb...,"{'text': array(['dry palms', 'wet palms', 'pal...",A,"['dry palms', 'wet palms', 'palms covered with...",Question: - George wants to warm his hands qui...,"""concepts"": [\n {\n ""name"": ""George's han..."
1,MCAS_2009_5_6516,Which of the following statements best explain...,{'text': array(['The refrigerator door is smoo...,B,"['The refrigerator door is smooth.', 'The refr...",Question: - Which of the following statements ...,"""concepts"": [\n {\n ""name"": ""Magnets"",\n ..."
2,Mercury_7233695,A fold observed in layers of sedimentary rock ...,"{'text': array(['cooling of flowing magma.', '...",B,"['cooling of flowing magma.', 'converging of c...",Question: - A fold observed in layers of sedim...,"""concepts"": [\n {\n ""name"": ""fold"",\n ..."
3,Mercury_7041615,Which of these do scientists offer as the most...,"{'text': array(['worldwide disease', 'global m...",D,"['worldwide disease', 'global mountain buildin...",Question: - Which of these do scientists offer...,"""concepts"": [\n {\n ""name"": ""asteroid imp..."
4,Mercury_7041860,A boat is acted on by a river current flowing ...,"{'text': array(['west', 'east', 'north', 'sout...",B,"['west', 'east', 'north', 'south']",Question: - A boat is acted on by a river curr...,"""concepts"": [\n {\n ""name"": ""River Curren..."


In [166]:
arc_mistral_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               1000 non-null   object
 1   question         1000 non-null   object
 2   choices          1000 non-null   object
 3   answerKey        1000 non-null   object
 4   options          1000 non-null   object
 5   prompt           1000 non-null   object
 6   prompt_response  1000 non-null   object
dtypes: object(7)
memory usage: 54.8+ KB


In [167]:
missing_concepts, missing_rel, missing_answer = 0, 0, 0
for idx, row in arc_mistral_df.iterrows():
    if row['id'] not in retained_arc_keys:
        continue
    if "\"concepts\"" not in row['prompt_response']:
        missing_concepts += 1
    
    if "\"relationships\"" not in row['prompt_response']:
        missing_rel += 1
        print(idx)
    
    if "\"answer\":" not in row['prompt_response']:
        missing_answer += 1
    print(row['prompt_response'].find("\"answer\":"))
    break
        

1345


In [168]:
missing_concepts, missing_rel, missing_answer

(0, 0, 0)

In [169]:
arc_mistral_df.iloc[0]['prompt_response'][1345:]

'"answer": "dry palms",\n"reasoning": "Rubbing dry palms against each other generates friction, which can help generate more heat due to the conversion of mechanical energy into thermal energy. W'

In [170]:
def process_response(response):
    ans_idx = response.find("\"answer\":")
    answer = ''

    if ans_idx != -1:
        answer = response[ans_idx:]
    else:
        ans_idx = len(response)
    
    rel_idx = response.find("\"relationships\":")
    relationships = ''

    if rel_idx != -1:
        relationships = response[rel_idx:ans_idx]
    else:
        rel_idx = len(response)

    concepts = response[:rel_idx]
    return preferred_response_tmplt.format(concepts, relationships, answer)


In [171]:
arc_mistral_df['response'] = arc_mistral_df['prompt_response'].apply(lambda r: process_response(r))
arc_mistral_df.head()

,id,question,choices,answerKey,options,prompt,prompt_response,response
0,Mercury_SC_415702,George wants to warm his hands quickly by rubb...,"{'text': array(['dry palms', 'wet palms', 'pal...",A,"['dry palms', 'wet palms', 'palms covered with...",Question: - George wants to warm his hands qui...,"""concepts"": [\n {\n ""name"": ""George's han...",All the main real world concepts essential to ...
1,MCAS_2009_5_6516,Which of the following statements best explain...,{'text': array(['The refrigerator door is smoo...,B,"['The refrigerator door is smooth.', 'The refr...",Question: - Which of the following statements ...,"""concepts"": [\n {\n ""name"": ""Magnets"",\n ...",All the main real world concepts essential to ...
2,Mercury_7233695,A fold observed in layers of sedimentary rock ...,"{'text': array(['cooling of flowing magma.', '...",B,"['cooling of flowing magma.', 'converging of c...",Question: - A fold observed in layers of sedim...,"""concepts"": [\n {\n ""name"": ""fold"",\n ...",All the main real world concepts essential to ...
3,Mercury_7041615,Which of these do scientists offer as the most...,"{'text': array(['worldwide disease', 'global m...",D,"['worldwide disease', 'global mountain buildin...",Question: - Which of these do scientists offer...,"""concepts"": [\n {\n ""name"": ""asteroid imp...",All the main real world concepts essential to ...
4,Mercury_7041860,A boat is acted on by a river current flowing ...,"{'text': array(['west', 'east', 'north', 'sout...",B,"['west', 'east', 'north', 'south']",Question: - A boat is acted on by a river curr...,"""concepts"": [\n {\n ""name"": ""River Curren...",All the main real world concepts essential to ...


### Winogrande

In [172]:
wino_mistral_df = pd.read_csv('../datasets/mistral/winogrande_train_with_prompt_and_response.csv')
wino_mistral_df.head()

,question,option1,option2,answer,prompt,id,prompt_response
0,John moved the couch from the garage to the ba...,garage,backyard,1,Question: - John moved the couch from the gara...,GO3wkLp9TC,"""concepts"": [\n {\n ""name"": ""garage"",\n ..."
1,The doctor diagnosed Justin with bipolar and R...,Justin,Robert,2,Question: - The doctor diagnosed Justin with b...,IPQZOkoLLz,"""concepts"": [\n {\n ""name"": ""Justin"",\n ..."
2,Dennis drew up a business proposal to present ...,Dennis,Logan,1,Question: - Dennis drew up a business proposal...,W85KDUn8gA,"""concepts"": [\n {\n ""name"": ""Dennis"",\n ..."
3,Felicia unexpectedly made fried eggs for break...,Felicia,Katrina,2,Question: - Felicia unexpectedly made fried eg...,UIUCGI68OY,"""concepts"": [\n {\n ""name"": ""Felicia"",\n ..."
4,My shampoo did not lather easily on my Afro ha...,shampoo,hair,2,Question: - My shampoo did not lather easily o...,WFAUCgvz91,"""concepts"": [\n {\n ""name"": ""shampoo"",\n ..."


In [173]:
wino_mistral_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   question         1000 non-null   object
 1   option1          1000 non-null   object
 2   option2          1000 non-null   object
 3   answer           1000 non-null   int64 
 4   prompt           1000 non-null   object
 5   id               1000 non-null   object
 6   prompt_response  1000 non-null   object
dtypes: int64(1), object(6)
memory usage: 54.8+ KB


In [174]:
missing_concepts, missing_rel, missing_answer = 0, 0, 0
for idx, row in wino_mistral_df.iterrows():
    if row['id'] not in retained_wino_keys:
        continue
    if "\"concepts\"" not in row['prompt_response']:
        missing_concepts += 1
    
    if "\"relationships\"" not in row['prompt_response']:
        missing_rel += 1
        print(idx)
    
    if "\"answer\":" not in row['prompt_response']:
        missing_answer += 1
    # print(row['prompt_response'].find("\"answer\":"))
    # break
        

In [175]:
missing_concepts, missing_rel, missing_answer

(0, 0, 76)

In [176]:
def process_response_wino(response):
    ans_idx = response.find("\"answer\":")
    answer = ''

    if ans_idx != -1:
        answer = response[ans_idx:]
    else:
        ans_idx = len(response)
    
    if len(answer) > 0 and answer.strip()[-1] == '}':
        answer = answer[:-1]
    
    rel_idx = response.find("\"relationships\":")
    relationships = ''

    if rel_idx != -1:
        relationships = response[rel_idx:ans_idx]
    else:
        rel_idx = len(response)

    if len(relationships) > 0 and relationships.strip()[-1] == '{':
        relationships = relationships[:-1]

    concepts = response[:rel_idx]
    return preferred_response_tmplt.format(concepts, relationships, answer)

In [177]:
wino_mistral_df['response'] = wino_mistral_df['prompt_response'].apply(lambda r: process_response_wino(r))
wino_mistral_df.head()

,question,option1,option2,answer,prompt,id,prompt_response,response
0,John moved the couch from the garage to the ba...,garage,backyard,1,Question: - John moved the couch from the gara...,GO3wkLp9TC,"""concepts"": [\n {\n ""name"": ""garage"",\n ...",All the main real world concepts essential to ...
1,The doctor diagnosed Justin with bipolar and R...,Justin,Robert,2,Question: - The doctor diagnosed Justin with b...,IPQZOkoLLz,"""concepts"": [\n {\n ""name"": ""Justin"",\n ...",All the main real world concepts essential to ...
2,Dennis drew up a business proposal to present ...,Dennis,Logan,1,Question: - Dennis drew up a business proposal...,W85KDUn8gA,"""concepts"": [\n {\n ""name"": ""Dennis"",\n ...",All the main real world concepts essential to ...
3,Felicia unexpectedly made fried eggs for break...,Felicia,Katrina,2,Question: - Felicia unexpectedly made fried eg...,UIUCGI68OY,"""concepts"": [\n {\n ""name"": ""Felicia"",\n ...",All the main real world concepts essential to ...
4,My shampoo did not lather easily on my Afro ha...,shampoo,hair,2,Question: - My shampoo did not lather easily o...,WFAUCgvz91,"""concepts"": [\n {\n ""name"": ""shampoo"",\n ...",All the main real world concepts essential to ...


### adding to cleaned df

In [178]:
arc_df_cleaned = arc_df_cleaned.merge(arc_mistral_df[['id', 'response']], on='id', how='left')
arc_df_cleaned.head()

,id,question,choices,answerKey,options,prompt,answer_text,chosen,response
0,Mercury_SC_415702,George wants to warm his hands quickly by rubb...,"{'text': array(['dry palms', 'wet palms', 'pal...",A,"['dry palms', 'wet palms', 'palms covered with...",Question: - George wants to warm his hands qui...,dry palms,All the main real world concepts essential to ...,All the main real world concepts essential to ...
1,MCAS_2009_5_6516,Which of the following statements best explain...,{'text': array(['The refrigerator door is smoo...,B,"['The refrigerator door is smooth.', 'The refr...",Question: - Which of the following statements ...,the refrigerator door contains iron.,All the main real world concepts essential to ...,All the main real world concepts essential to ...
2,Mercury_7233695,A fold observed in layers of sedimentary rock ...,"{'text': array(['cooling of flowing magma.', '...",B,"['cooling of flowing magma.', 'converging of c...",Question: - A fold observed in layers of sedim...,converging of crustal plates.,All the main real world concepts essential to ...,All the main real world concepts essential to ...
3,Mercury_7041615,Which of these do scientists offer as the most...,"{'text': array(['worldwide disease', 'global m...",D,"['worldwide disease', 'global mountain buildin...",Question: - Which of these do scientists offer...,impact of an asteroid created dust that blocke...,All the main real world concepts essential to ...,All the main real world concepts essential to ...
4,Mercury_7041860,A boat is acted on by a river current flowing ...,"{'text': array(['west', 'east', 'north', 'sout...",B,"['west', 'east', 'north', 'south']",Question: - A boat is acted on by a river curr...,east,All the main real world concepts essential to ...,All the main real world concepts essential to ...


In [181]:
print(arc_df_cleaned.iloc[0][['chosen', 'response']].values)

['All the main real world concepts essential to answer the question are: \n{\n  "concepts": [\n    {\n      "name": "Friction",\n      "description": "The resistance that one surface or object encounters when moving over another."\n    },\n    {\n      "name": "Heat Generation",\n      "description": "The production of heat energy resulting from processes like friction."\n    },\n    {\n      "name": "Dry Palms",\n      "description": "Skin surfaces of the palms without any moisture or lubrication."\n    },\n    {\n      "name": "Wet Palms",\n      "description": "Skin surfaces of the palms with moisture such as water."\n    },\n    {\n      "name": "Palms covered with Oil",\n      "description": "Skin surfaces of the palms coated with a thin layer of oil."\n    },\n    {\n      "name": "Palms covered with Lotion",\n      "description": "Skin surfaces of the palms coated with a layer of lotion."\n    }\n  ]\n}\n\nIn terms of ontology, the relationship for each concept to the other conc

In [183]:
arc_df_cleaned.rename(columns={'response': 'rejected'}, inplace=True)
arc_df_cleaned.head()

,id,question,choices,answerKey,options,prompt,answer_text,chosen,rejected
0,Mercury_SC_415702,George wants to warm his hands quickly by rubb...,"{'text': array(['dry palms', 'wet palms', 'pal...",A,"['dry palms', 'wet palms', 'palms covered with...",Question: - George wants to warm his hands qui...,dry palms,All the main real world concepts essential to ...,All the main real world concepts essential to ...
1,MCAS_2009_5_6516,Which of the following statements best explain...,{'text': array(['The refrigerator door is smoo...,B,"['The refrigerator door is smooth.', 'The refr...",Question: - Which of the following statements ...,the refrigerator door contains iron.,All the main real world concepts essential to ...,All the main real world concepts essential to ...
2,Mercury_7233695,A fold observed in layers of sedimentary rock ...,"{'text': array(['cooling of flowing magma.', '...",B,"['cooling of flowing magma.', 'converging of c...",Question: - A fold observed in layers of sedim...,converging of crustal plates.,All the main real world concepts essential to ...,All the main real world concepts essential to ...
3,Mercury_7041615,Which of these do scientists offer as the most...,"{'text': array(['worldwide disease', 'global m...",D,"['worldwide disease', 'global mountain buildin...",Question: - Which of these do scientists offer...,impact of an asteroid created dust that blocke...,All the main real world concepts essential to ...,All the main real world concepts essential to ...
4,Mercury_7041860,A boat is acted on by a river current flowing ...,"{'text': array(['west', 'east', 'north', 'sout...",B,"['west', 'east', 'north', 'south']",Question: - A boat is acted on by a river curr...,east,All the main real world concepts essential to ...,All the main real world concepts essential to ...


In [184]:
wino_df_cleaned = wino_df_cleaned.merge(wino_mistral_df[['id', 'response']], on='id', how='left')
wino_df_cleaned.head()

,id,question,option1,option2,answer,prompt,answer_text,chosen,response
0,GO3wkLp9TC,John moved the couch from the garage to the ba...,garage,backyard,1,Question: - John moved the couch from the gara...,garage,All the main real world concepts essential to ...,All the main real world concepts essential to ...
1,IPQZOkoLLz,The doctor diagnosed Justin with bipolar and R...,Justin,Robert,2,Question: - The doctor diagnosed Justin with b...,robert,All the main real world concepts essential to ...,All the main real world concepts essential to ...
2,W85KDUn8gA,Dennis drew up a business proposal to present ...,Dennis,Logan,1,Question: - Dennis drew up a business proposal...,dennis,All the main real world concepts essential to ...,All the main real world concepts essential to ...
3,UIUCGI68OY,Felicia unexpectedly made fried eggs for break...,Felicia,Katrina,2,Question: - Felicia unexpectedly made fried eg...,katrina,All the main real world concepts essential to ...,All the main real world concepts essential to ...
4,WFAUCgvz91,My shampoo did not lather easily on my Afro ha...,shampoo,hair,2,Question: - My shampoo did not lather easily o...,hair,All the main real world concepts essential to ...,All the main real world concepts essential to ...


In [185]:
wino_df_cleaned.rename(columns={'response': 'rejected'}, inplace=True)
wino_df_cleaned.head()

,id,question,option1,option2,answer,prompt,answer_text,chosen,rejected
0,GO3wkLp9TC,John moved the couch from the garage to the ba...,garage,backyard,1,Question: - John moved the couch from the gara...,garage,All the main real world concepts essential to ...,All the main real world concepts essential to ...
1,IPQZOkoLLz,The doctor diagnosed Justin with bipolar and R...,Justin,Robert,2,Question: - The doctor diagnosed Justin with b...,robert,All the main real world concepts essential to ...,All the main real world concepts essential to ...
2,W85KDUn8gA,Dennis drew up a business proposal to present ...,Dennis,Logan,1,Question: - Dennis drew up a business proposal...,dennis,All the main real world concepts essential to ...,All the main real world concepts essential to ...
3,UIUCGI68OY,Felicia unexpectedly made fried eggs for break...,Felicia,Katrina,2,Question: - Felicia unexpectedly made fried eg...,katrina,All the main real world concepts essential to ...,All the main real world concepts essential to ...
4,WFAUCgvz91,My shampoo did not lather easily on my Afro ha...,shampoo,hair,2,Question: - My shampoo did not lather easily o...,hair,All the main real world concepts essential to ...,All the main real world concepts essential to ...


In [187]:
print(wino_df_cleaned.iloc[0][['chosen', 'rejected']].values)

['All the main real world concepts essential to answer the question are: \n{\n  "concepts": [\n    {\n      "name": "John",\n      "description": "A person who performs the action of moving the couch."\n    },\n    {\n      "name": "couch",\n      "description": "A large piece of furniture for seating, subject of the moving action."\n    },\n    {\n      "name": "garage",\n      "description": "A place for storing vehicles, initially storing the couch."\n    },\n    {\n      "name": "backyard",\n      "description": "An outdoor area behind a house, destination of the moved couch."\n    },\n    {\n      "name": "space creation",\n      "description": "The objective of moving the couch from the garage."\n    }\n  ]\n}\n\nIn terms of ontology, the relationship for each concept to the other concepts in the context of the sentence and the question are:\n{\n  "relationships": [\n    {\n      "concept": "John",\n      "relationship": "moves",\n      "related_concept": "couch",\n      "descrip

In [188]:
print(wino_df_cleaned.iloc[0][['prompt']].values)

['Question: - John moved the couch from the garage to the backyard to create space. The _ is small.\nOptions: - [\'garage\', \'backyard\']\n\nAll the main real world concepts essential to answer the question are: \nList them down along with one line description\nRemember to return the response in a json structure as defined below - \n\n"concepts": [\n  {\n    "name": "concept/entity name",\n    "description": "one line description"\n  },\n  {\n    "name": "conecpt/entity name",\n    "description": "one line description"\n  },\n  ...\n]\n\n[your concepts list]\n\nIn manner of ontology, describe the relationship for each concept to the other concepts in the context of the sentence in a structured manner along with one line description for each are:\nRemember to return the response in a json structure as defined below -\n\n"relationships": [\n\t{\n\t\t"concept": "concept1",\n\t\t"relationship": "relation",\n\t\t"related_concept": "concept2",\n\t\t"description": "one line description"\n\t}

In [189]:
def get_min_prompt(prompt):
    end_idx = prompt.find('\n\nAll the main real world concepts essential')
    return prompt[:end_idx]

In [191]:
arc_df_cleaned['min_prompt'] = arc_df_cleaned['prompt'].apply(lambda x: get_min_prompt(x))
wino_df_cleaned['min_prompt'] = wino_df_cleaned['prompt'].apply(lambda x: get_min_prompt(x))

In [196]:
print(wino_df_cleaned.iloc[30]['min_prompt'])

Question: - The car gripped the asphalt better than the truck although the _ had poor traction.
Options: - ['car', 'truck']


In [197]:
print(arc_df_cleaned.iloc[30]['min_prompt'])

Question: - Which of following is a property shared by the elements in the carbon family?
Options: - ['an atomic number of 6', 'an atomic mass of 12', 'the same electron configuration', 'the number of valence electrons']


# Combining both datasets

In [198]:
arc_df_min = arc_df_cleaned[['id', 'prompt', 'answer_text', 'chosen', 'rejected']].copy()
arc_df_min['source'] = 'arc'

wino_df_min = wino_df_cleaned[['id', 'prompt', 'answer_text', 'chosen', 'rejected']].copy()
wino_df_min['source'] = 'wino'

In [199]:
arc_df_min.head()

,id,prompt,answer_text,chosen,rejected,source
0,Mercury_SC_415702,Question: - George wants to warm his hands qui...,dry palms,All the main real world concepts essential to ...,All the main real world concepts essential to ...,arc
1,MCAS_2009_5_6516,Question: - Which of the following statements ...,the refrigerator door contains iron.,All the main real world concepts essential to ...,All the main real world concepts essential to ...,arc
2,Mercury_7233695,Question: - A fold observed in layers of sedim...,converging of crustal plates.,All the main real world concepts essential to ...,All the main real world concepts essential to ...,arc
3,Mercury_7041615,Question: - Which of these do scientists offer...,impact of an asteroid created dust that blocke...,All the main real world concepts essential to ...,All the main real world concepts essential to ...,arc
4,Mercury_7041860,Question: - A boat is acted on by a river curr...,east,All the main real world concepts essential to ...,All the main real world concepts essential to ...,arc


In [200]:
wino_df_min.head()

,id,prompt,answer_text,chosen,rejected,source
0,GO3wkLp9TC,Question: - John moved the couch from the gara...,garage,All the main real world concepts essential to ...,All the main real world concepts essential to ...,wino
1,IPQZOkoLLz,Question: - The doctor diagnosed Justin with b...,robert,All the main real world concepts essential to ...,All the main real world concepts essential to ...,wino
2,W85KDUn8gA,Question: - Dennis drew up a business proposal...,dennis,All the main real world concepts essential to ...,All the main real world concepts essential to ...,wino
3,UIUCGI68OY,Question: - Felicia unexpectedly made fried eg...,katrina,All the main real world concepts essential to ...,All the main real world concepts essential to ...,wino
4,WFAUCgvz91,Question: - My shampoo did not lather easily o...,hair,All the main real world concepts essential to ...,All the main real world concepts essential to ...,wino


In [205]:
concept_df = pd.concat([arc_df_min, wino_df_min], axis=0).sample(frac=1.0).reset_index(drop=True)
concept_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1747 entries, 0 to 1746
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           1747 non-null   object
 1   prompt       1747 non-null   object
 2   answer_text  1747 non-null   object
 3   chosen       1747 non-null   object
 4   rejected     1747 non-null   object
 5   source       1747 non-null   object
dtypes: object(6)
memory usage: 82.0+ KB


In [206]:
concept_df.head()

,id,prompt,answer_text,chosen,rejected,source
0,Mercury_SC_415737,Question: - A small ice cube at a temperature ...,between 0°c and 28°c,All the main real world concepts essential to ...,All the main real world concepts essential to ...,arc
1,NYSEDREGENTS_2004_4_8,Question: - Base your answers on the informati...,slower,All the main real world concepts essential to ...,All the main real world concepts essential to ...,arc
2,sLCexZmq40,Question: - Nick's favorite color is black and...,aaron,All the main real world concepts essential to ...,All the main real world concepts essential to ...,wino
3,MSA_2013_5_44,Question: - A ship leaks a large amount of oil...,water birds will be unable to use their wings.,All the main real world concepts essential to ...,All the main real world concepts essential to ...,arc
4,detlbvDJvm,Question: - Kyle stole the ball from Ryan duri...,kyle,All the main real world concepts essential to ...,All the main real world concepts essential to ...,wino


In [207]:
concept_df.to_csv('../datasets/fine-tuning/concept_aware_reasoning_preference_data.csv', index=False)